# COVID-19 Mortality in the United States and Canada #

In [ ]:
%pip import oracledb
%pip install altair
#Importing packages
import pandas as pd
import altair as alt
import oracledb 
import csv

In [ ]:
#Loading the USA and Canada datasets
data_USA = pd.read_csv("data_USA.csv")
data_CA = pd.read_csv("data_CA.csv")

data_CA.head()
data_USA.head()

### Data Wrangling Canadian Dataset ###
- Since the Canadian dataset contains various COVID-19 status, we are only interested in using "deaths" for calculating mortality, we will be filtering rows with deaths status.
- Now that the dataset only contain deaths cases, we will be dropping "rate_per_100000" and "status" columns. 
- We need to convert "date" to year and filter out 2024 since we are only interested in the years during the COVID-19 outbreak.
- To avoid overlapping information, we will also be removing entries where "gender" and "age_group" is "all"
- Lastly, we will rename the columns for clarity

In [ ]:
# Filter rows where "status" contains "deaths"
data_CA_Pre = data_CA[data_CA["status"].str.contains("deaths", na=False)].copy()

# Drop unnecessary columns
data_CA_Pre.drop(columns=["rate_per_100000", "status"], inplace=True)

# Convert "date" to year and filter out 2024
data_CA_Pre["date"] = pd.to_datetime(data_CA_Pre["date"]).dt.year
data_CA_Pre = data_CA_Pre[data_CA_Pre["date"] != 2024]
data_CA_Pre["date"] = data_CA_Pre["date"].astype("category")

# Remove rows where "gender" or "age_group" is "all"
data_CA_Pre = data_CA_Pre[(data_CA_Pre["gender"] != "all") & (data_CA_Pre["age_group"] != "all")]

# Rename columns for clarity
data_CA_Cleaned = data_CA_Pre.rename(columns={"date": "year", "count": "deaths"})

# Display the cleaned DataFrame
data_CA_Cleaned

### Data Wrangling USA Dataset ###
- We will be dropping unnecessary columns including overlapping informations
- Then we will rename the columns to match what we have in the Canadian dataset
- Since the amount of NAs in this dataset are negligible, which do not affect our interpretation of the question, we decided to remove them. 
- To make it similar to the Canadian dataset, we need to convert "year" into datetime and extract year as an integer then change it into a categorical datatype. We think that categorical data is more memory-efficent for repeated values to be treated as discrete groups and not continuous numbers.
- Then we will remove the rows with "all sexes" and "all age" to prevent duplication.
- Converting gender and deaths into the similar datatypes as the Canadian dataset
- We only keep relevant columns, which are the same as those in the Canadian dataset.


# Drop unnecessary columns
data_USA_Pre = data_USA.drop(
    columns=[
        "Data As Of", "Start Date", "End Date", "Group", "Month", "State",
        "Total Deaths", "Pneumonia Deaths", "Pneumonia and COVID-19 Deaths",
        "Influenza Deaths", "Pneumonia, Influenza, or COVID-19 Deaths", "Footnote"
    ]
).copy()

# Rename columns for clarity
data_USA_Pre.rename(columns={
    "Year": "year",
    "Sex": "gender",
    "Age Group": "age_group",
    "COVID-19 Deaths": "deaths"
}, inplace=True)

# Drop rows with missing "deaths" or "year"
data_USA_Pre.dropna(subset=["deaths", "year"], inplace=True)

# Convert "year" to integer
data_USA_Pre["year"] = pd.to_datetime(data_USA_Pre["year"], format="%Y").dt.year
data_USA_Pre["year"] = data_USA_Pre["year"].astype("category")

# Remove rows where gender is "All Sexes" and age_group is "All Ages"
data_USA_Pre = data_USA_Pre[(data_USA_Pre["gender"] != "All Sexes") &
                            (data_USA_Pre["age_group"] != "All Ages")]

# Change gender categories to lowercase
data_USA_Pre["gender"] = data_USA_Pre["gender"].str.lower()

# Convert "deaths" to int64
data_USA_Pre["deaths"] = data_USA_Pre["deaths"].astype("int64")

# Keep only relevant columns
data_USA_Cleaned = data_USA_Pre[["year", "age_group", "gender", "deaths"]]

# Display cleaned DataFrame
data_USA_Cleaned

- Since both datasets are structured differently where each of them as a different age-grouping methods, we decided to put create overlapping groups to better merge these two datasets into the database. 
- Mapped different age group formats to common age bins:
0 to 29 years
30 to 49 years
50 + years
- Applied consistent age bins using .replace()
- After cleaning, the dataset was more structured, making it easier to analyze trends.


In [ ]:
# Define broad age bins using dictionary comprehension
age_bin_mapping = {
    **dict.fromkeys(["0-17 years", "18-29 years"], "0 to 29"),
    **dict.fromkeys(["30-39 years", "40 to 49"], "30 to 49"),
    **dict.fromkeys(["50-64 years","65-74 years", "75-84 years", "85 years and over" ], "50 +")}

# Apply transformation to both datasets in one step
for df in [data_USA_Cleaned]:
    df["age_group"] = df["age_group"].str.strip().str.lower().replace(age_bin_mapping)


# Define broad age bins using dictionary comprehension
age_bin_mapping = {
    **dict.fromkeys(['20 to 29', '0 to 11', '12 to 19'], "0 to 29"),
    **dict.fromkeys([ '30 to 39', '40 to 49'], "30 to 49"),
    **dict.fromkeys(['50 to 59', '60 to 69', '70 to 79', '80+'], "50 +")}

# Apply transformation to both datasets in one step
for df in [data_CA_Cleaned]:
    df["age_group"] = df["age_group"].str.strip().str.lower().replace(age_bin_mapping)

#Remove overlapping age_groups to make unified age_group categories. 

values_to_remove = ['under 1 year',  '1-4 years', '5-14 years',
       '15-24 years', '25-34 years',  '35-44 years',
       '40-49 years', '45-54 years', '55-64 years']
data_USA_Cleaned = data_USA_Cleaned[~data_USA_Cleaned['age_group'].isin(values_to_remove)]

Now that both datasets are cleaned, let's export them to csv files, ready to be transformed into insert statements to the database.

In [ ]:
data_CA_Cleaned.to_csv('data_CA_Cleaned.csv', index=False)
files.download('data_CA_Cleaned.csv')

data_USA_Cleaned.to_csv('data_USA_Cleaned.csv', index=False)
files.download('data_USA_Cleaned.csv')


### Exploratory Data Analysis ###

- The two datasets here contain information regarding deaths due to COVID-19 in Canada and the USA, categorized by year, age group, and gender.
The EDA was performed on both raw (uncleaned) and cleaned data to compare differences and generate meaningful insights.


In [ ]:
#1. Faceted by gender over the years for death count

chart_CA = alt.Chart(data_CA_Cleaned, title="Death count in Canada due to Covid-19 over the years by Gender").mark_bar().encode(
    x=alt.X("year:O", title="Year"),
    y=alt.Y("sum(deaths):Q", title="Total Death Count"),
    color=alt.Color("gender:N", title="Gender"))

chart_USA = alt.Chart(data_USA_Cleaned, title="Death count in USA due to Covid-19 over the years by Gender").mark_bar().encode(
    x=alt.X("year:O", title="Year"),
    y=alt.Y("sum(deaths):Q", title="Total Death Count"),
    color=alt.Color("gender:N", title="Gender"))

chart_CA|chart_USA

**At first glance:**
- The total number of deaths in Canada vs. the USA from 2020 to 2023.
- Death counts are further split by gender to observe any disparities.


**Key Insights:**
- The USA has significantly more deaths compared to Canada, likely due primarily due to a larger population size and differing COVID-19 policies.
- Males had consistently higher death counts than females in both countries.
- Deaths peaked in 2021 for the USA, after which they declined, possibly due to the introduction of vaccination.
- Canada registered its largest death count in 2022, its highest since the start of the pandemic in 2020, largely due to a new COVID-19 variant which was highly transmissible and due to reduced restrictions and masking requirements. (Statistics Canada, 2023)


In [ ]:
chart_CA2 = alt.Chart(data_CA_Cleaned, title = "Sum of Deaths in Canada due to Covid-19 over the years").mark_rect().encode(
    x=alt.X("age_group:N", title="age_groups"),
    y=alt.Y("year:N", title="Year" ),
 tooltip=alt.Tooltip(["sum(deaths):Q"], title="Total Deaths"),
    color=alt.Color("sum(deaths):Q", title="Death Count")
).transform_aggregate(
    deaths="sum(deaths)",  # Aggregate death counts by year
    groupby=["year", "age_group"]  # Group by year and age group
).properties(width=200, height=200)

chart_USA2 = alt.Chart(data_USA_Cleaned, title="Sum of Deaths  in USA due to Covid-19 over the years").mark_rect().encode(
    x=alt.X("age_group:N", title="Age Group"),
    y=alt.Y("year:N", title="Year"),
    tooltip=alt.Tooltip(["sum(deaths):Q"], title="Total Deaths"),
    color=alt.Color("sum(deaths):Q", title="Death Count")
).transform_aggregate(
    deaths="sum(deaths)",  # Aggregate death counts by year
    groupby=["year", "age_group"]  # Group by year and age group
).properties(width=200, height=200)


chart_CA2

**At first glance:** 

- A heatmap where color intensity represents death counts.
- X-axis: Age Groups, Y-axis: Years.
- Darker colors = higher deaths.


**Key Insights:**
- Shows similar patterns to the bar chart above. 
- Younger age groups had very few deaths, confirming the lower severity of COVID-19 in children.
- The deadliest years in Canada were 2021 and 2022 (variant-driven waves), while 2020 and 2021 were the deadliest years in the USA (initial pandemic waves), which aligns with pandemic waves.
- Both countries saw a decline in deaths in 2023, likely due to the increased accessibility and availability of vaccines and improved treatments.


In [ ]:
#Faceted by country over the years for death count

chart_CA1 =  alt.Chart(data_CA_Cleaned, title="Death count in Canada due to Covid-19 for different age groups, colored by year").mark_bar().encode(
    x=alt.X("age_group:N", title="Age Groups"),
    y=alt.Y("sum(deaths):Q", title="Total Death Count"),
    color=alt.Color("year:N")
    ).properties(width=400, height=400)


chart_USA1 =  alt.Chart(data_USA_Cleaned, title="Death count in USA due to Covid-19 for different age group, colored by year").mark_bar().encode(
    x=alt.X("age_group:N", title="Age Groups"),
    y=alt.Y("sum(deaths):Q", title="Total Death Count"),
    color=alt.Color("year:N")
).properties(width=400, height=400)


chart_CA1|chart_USA1

**At first glance:**
- The distribution of deaths across different age groups in Canada and the USA propotioned by year.

**Key Insights:**

- Older age groups (50+) had the highest number of deaths, reinforcing that COVID-19 was more severe for older individuals and that older individuals had the highest fatality rate.
- Younger age groups (0-29) had the lowest deaths, supporting research that younger people were at lower risk.
- Similar age-group trends were observed in both countries, signifying that despite different healthcare systems, age-based risk factors remain similar.

**Conclusion:**
- The cleaned dataset provided clearer insights compared to the raw data
- The USA had a much higher death count than Canada, and older populations were most affected.
- Males had consistently higher death rates than females across all years.
- 2021 and 2022 were the deadliest years, with a sharp decline afterward due to vaccines and improved healthcare.


- The results from our EDA have proven that the cleaned and processed data is well-structured for our analysis, so no major changes are required to handle the data. However, the following adjustments will help improve our analysis:

    -  **Gender-Based Analysis:** Since male deaths are consistently higher across both countries, we will explore whether this trend holds across specific age groups and years. 
    - **Peak Years Insight:** Since the highest death rates occurred in 2021 (USA) and 2022 (Canada), we will focus on analyzing the reasons behind these peaks (e.g., accessibility of vaccines, new variants, policy surrounding handling of pandemic, etc).

## Extracting Insert Statements ##

In [ ]:
# Input CSV file
csv_file = '/Users/Downloads/data_USA_cleaned.csv'

# Output file for SQL INSERT statements
output_file = '/Users/Downloads/USA_insert_statements.sql'

with open(csv_file, mode='r') as file:
    csv_reader = csv.DictReader(file) 
    with open(output_file, mode='w') as sql_file:
        for row in csv_reader:
            year = row['year']
            age_group = row['age_group']
            gender = row['gender']
            deaths = row['deaths']
            
            # Generate the repeating INSERT statements:
            insert_statement = (
                f"INSERT INTO total_deaths (year, age_group, gender, deaths, country) "
                f"VALUES ({year}, '{age_group}', '{gender}', {deaths}, 'USA');\n"
            )
            
            sql_file.write(insert_statement)

In [ ]:
# Input CSV file
csv_file = '/Users/Downloads/data_CA_cleaned.csv'

# Output file for SQL INSERT statements
output_file = '/Users/Downloads/CA_insert_statements.sql'

with open(csv_file, mode='r') as file:
    csv_reader = csv.DictReader(file) 
    with open(output_file, mode='w') as sql_file:
        for row in csv_reader:
            year = row['year']
            age_group = row['age_group']
            gender = row['gender']
            deaths = row['deaths']
            
            # Generate the repeating INSERT statements:
            insert_statement = (
                f"INSERT INTO total_deaths (year, age_group, gender, deaths, country) "
                f"VALUES ({year}, '{age_group}', '{gender}', {deaths}, 'CA');\n"
            )
            
            sql_file.write(insert_statement)

## It's time for the Database!! ##

In [ ]:
# Make a function to run database query using cursor
def execute_db(query):
    dsn = oracledb.makedsn("localhost", 1522, service_name="stu")
    #Note that I've created these table in my oracle database, we can use change the user and password to create your own version
    connection = oracledb.connect(user="_", password="_", dsn=dsn)
    cursor = connection.cursor()
    cursor.execute(query)
    connection.commit()
    print("Query executed sucessfully")

In [ ]:
def drop_tables():
    tables = [
        "canada_deaths",
        "usa_deaths",
        "aggregated_deaths",
        "total_deaths"]
    dsn = oracledb.makedsn("localhost", 1522, service_name="stu")
    connection = oracledb.connect(user="_", password="_", dsn=dsn)
    cursor = connection.cursor()
    for table in tables:
        try:
            execute_db(f"DROP TABLE {table} CASCADE CONSTRAINTS")
            print(f"Dropped table: {table}")
        except Exception as e:
            print("Error")
            continue  

In [ ]:
#Defining the tables we need to create in the database
create_total_deaths = """
CREATE TABLE total_deaths (
    year NUMBER NOT NULL,
    age_group VARCHAR2(20) NOT NULL,
    gender VARCHAR2(10) NOT NULL,
    deaths NUMBER NOT NULL,
    country VARCHAR2(50) NOT NULL
)
"""

create_aggregated_deaths = """
CREATE TABLE aggregated_deaths (
    year NUMBER NOT NULL,
    age_group VARCHAR2(20) NOT NULL,
    gender VARCHAR2(20) NOT NULL,
    total_deaths NUMBER,
    country VARCHAR2(50) NOT NULL,
    PRIMARY KEY (year, age_group, gender, country)
)
"""

create_canada_deaths = """
CREATE TABLE canada_deaths (
    year NUMBER,
    age_group VARCHAR2(50),
    gender VARCHAR2(10),
    deaths NUMBER,
    country VARCHAR2(50),
    PRIMARY KEY (year, age_group, gender, country),
    FOREIGN KEY (year, age_group, gender, country)
    REFERENCES aggregated_deaths(year, age_group, gender, country)
)
"""

create_usa_deaths = """
CREATE TABLE usa_deaths (
    year NUMBER,
    age_group VARCHAR2(50),
    gender VARCHAR2(10),
    deaths NUMBER,
    country VARCHAR2(50),
    PRIMARY KEY (year, age_group, gender, country),
    FOREIGN KEY (year, age_group, gender, country)
    REFERENCES aggregated_deaths(year, age_group, gender, country)
)
"""

In [ ]:
# A Python function to read the sql file of insert statements
def file_reader(file):
    dsn = oracledb.makedsn("localhost", 1522, service_name="stu")
    #Note that I've created these table in my oracle database, we can use change the user and password to create your own version
    connection = oracledb.connect(user="_", password = "_", dsn=dsn)
    cursor = connection.cursor()
    
    with open(file, 'r') as sql_file:
        sql_content = sql_file.read()
        
        statements = [query.strip() for query in sql_content.split(';')]
        for query in statements:
            cursor.execute(query)
            
        connection.commit()
        print("Loaded successfully")
        cursor.close()
        connection.close()

In [ ]:
drop_tables()
execute_db(create_total_deaths)
file_reader("CA_insert_statements.sql") #takes about 3 mins
file_reader("USA_insert_statements.sql") #takes about 24-30 mins

In [ ]:
#check to see if the data are loaded 
dsn = oracledb.makedsn("localhost", 1522, service_name="stu")
#Note that I've created these table in my oracle database, we can use change the user and password to create your own version
connection = oracledb.connect(user="_", password="_", dsn=dsn)

cur = connection.cursor()
for row in cur.execute("select * from total_deaths"):
    print(row)
cur.close()

In [ ]:
#Create aggegated_deaths table
execute_db(create_aggregated_deaths)
#Populate data into aggregated_deaths table
execute_db("""
INSERT INTO aggregated_deaths (year, age_group, gender, total_deaths, country)
SELECT
    year,
    age_group,
    gender,
    SUM(deaths) AS total_deaths,
    country
FROM
    total_deaths
GROUP BY
    year,
    age_group,
    gender,
    country
""")

#Create canada_deaths table in the database
execute_db(create_canada_deaths)
#Create usa_deaths table in the database
execute_db(create_usa_deaths)
#Populate data into canada_deaths table
execute_db("""
INSERT INTO canada_deaths (year, age_group, gender, deaths, country)
SELECT year, age_group, gender, total_deaths, country
FROM aggregated_deaths ad
WHERE ad.country = 'Canada'
""")
#Populate data into usa_deaths table
execute_db("""INSERT INTO usa_deaths (year, age_group, gender, deaths, country)
SELECT year, age_group, gender, total_deaths, country
FROM aggregated_deaths ad
WHERE ad.country = 'USA'
""")

In [ ]:
# USA's data #Create another table to store the estimated population for each year and each age group
execute_db("""
CREATE TABLE usa_population (
    YEAR INT,
    AGE_GROUP VARCHAR(10),
    POPULATION INT
)
""")

#Populate the estimated population found the *document file* into the newly created table
queries = [
    "INSERT INTO usa_population VALUES (2020, '0 to 29', 254210460)",
    "INSERT INTO usa_population VALUES (2020, '30 to 49', 172198146)",
    "INSERT INTO usa_population VALUES (2020, '50 +', 236593554)",
    "INSERT INTO usa_population VALUES (2021, '0 to 29', 252095620)",
    "INSERT INTO usa_population VALUES (2021, '30 to 49', 172575984)",
    "INSERT INTO usa_population VALUES (2021, '50 +', 239115886)",
    "INSERT INTO usa_population VALUES (2022, '0 to 29', 251414856)",
    "INSERT INTO usa_population VALUES (2022, '30 to 49', 173851316)",
    "INSERT INTO usa_population VALUES (2022, '50 +', 241276650)",
    "INSERT INTO usa_population VALUES (2023, '0 to 29', 250806986)",
    "INSERT INTO usa_population VALUES (2023, '30 to 49', 175463718)",
    "INSERT INTO usa_population VALUES (2023, '50 +', 243559086)"
]

for query in queries:
    execute_db(query)


#Calulate the mortality rate, joining both tables
dsn = oracledb.makedsn("localhost", 1522, service_name="stu")
connection = oracledb.connect(user="-", password="-", dsn=dsn)
cursor = connection.cursor()

query = """
SELECT 
    d.YEAR,
    d.AGE_GROUP,
    d.TOTAL_DEATHS,
    p.POPULATION,
    (d.TOTAL_DEATHS * 100000.0 / p.POPULATION) AS MORTALITY_RATE
FROM 
    (SELECT YEAR, AGE_GROUP, SUM(DEATHS) AS TOTAL_DEATHS
     FROM usa_deaths
     GROUP BY YEAR, AGE_GROUP) d
JOIN 
    usa_population p ON d.YEAR = p.YEAR AND d.AGE_GROUP = p.AGE_GROUP
ORDER BY 
    d.YEAR, d.AGE_GROUP
"""

usa_w_population = pd.read_sql(query,connection)
connection.close()
print(usa_w_population)

### Canada's turn ###

In [ ]:
# #Create another table to store the estimated population for each year and each age group
execute_db("""
CREATE TABLE ca_population (
    YEAR INT,
    AGE_GROUP VARCHAR(10),
    POPULATION INT
)
""")

#Populate the estimated population found the *document file* into the newly created table
queries = [
    "INSERT INTO ca_population VALUES (2020, '0 to 29', 13332834)",
    "INSERT INTO ca_population VALUES (2020, '30 to 49', 10131696)",
    "INSERT INTO ca_population VALUES (2020, '50 +', 14564108)",
    "INSERT INTO ca_population VALUES (2021, '0 to 29', 13228492)",
    "INSERT INTO ca_population VALUES (2021, '30 to 49', 10234175)",
    "INSERT INTO ca_population VALUES (2021, '50 +', 14777197)",
    "INSERT INTO ca_population VALUES (2022, '0 to 29', 13469283)",
    "INSERT INTO ca_population VALUES (2022, '30 to 49', 10473634)",
    "INSERT INTO ca_population VALUES (2022, '50 +', 14993017)",
    "INSERT INTO ca_population VALUES (2023, '0 to 29', 13923612)",
    "INSERT INTO ca_population VALUES (2023, '30 to 49', 10938795)",
    "INSERT INTO ca_population VALUES (2023, '50 +', 15221077)"
]

for query in queries:
    execute_db(query)

#Calulate the mortality rate, joining both tables
dsn = oracledb.makedsn("localhost", 1522, service_name="stu")
connection = oracledb.connect(user="-", password="-", dsn=dsn)
cursor = connection.cursor()

cursor.execute("""
SELECT 
    d.YEAR,
    d.AGE_GROUP,
    d.TOTAL_DEATHS,
    p.POPULATION,
    (d.TOTAL_DEATHS * 100000.0 / p.POPULATION) AS MORTALITY_RATE
FROM 
    (SELECT YEAR, AGE_GROUP, SUM(DEATHS) AS TOTAL_DEATHS
     FROM canada_deaths
     GROUP BY YEAR, AGE_GROUP) d
JOIN 
    ca_population p ON d.YEAR = p.YEAR AND d.AGE_GROUP = p.AGE_GROUP
ORDER BY 
    d.YEAR, d.AGE_GROUP
""")
for row in cursor:
    print(row)
cursor.close()
connection.close()

Now that we have all the information we need to answer our questions, let's export these information and make good visualization to help us answer our research questions.

In [ ]:
#Export this csv for visualizations
ca_w_population.to_csv('ca_w_population.csv')
usa_w_population.to_csv('usa_w_population.csv')

In [ ]:
#Exporting Canadian information into csv file. 
dsn = oracledb.makedsn("localhost", 1522, service_name="stu")
connection = oracledb.connect(user="_", password="_", dsn=dsn)
query = """
    SELECT year, age_group, gender, deaths, country
    FROM canada_deaths
    ORDER BY year ASC, age_group
"""

canada_df = pd.read_sql(query,connection)
connection.close()
print(canada_df)

canada_df.to_csv('canada_deaths_data.csv')

In [ ]:
#Exporting US information into csv file. 
dsn = oracledb.makedsn("localhost", 1522, service_name="stu")
connection = oracledb.connect(user="_", password="_", dsn=dsn)
query = """
    SELECT year, age_group, gender, deaths, country
    FROM usa_deaths
    ORDER BY year ASC, age_group
"""

usa_df = pd.read_sql(query,connection)
connection.close()
print(usa_df)

usa_df.to_csv('usa_deaths_data.csv')